# Inspect lenses and generate GLASS configs

## Imports

In [ ]:
import sys
import os
import pprint
import json
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata import Cutout2D

root = "/Users/phdenzel/gleam"
sys.path.append(root)
import gleam
from gleam.lensobject import LensObject
from gleam.multilens import MultiLens
from gleam.utils.plotting import IPColorbar, IPPointCache
import gleam.utils.colors as gcl
gcl.GLEAMcmaps.register_all()

## Main

### Search for a fits file

In [ ]:
fidx = 19
files = !ls data/delay_qsos/*.fits
pprint.pprint(files)
filename = files[fidx]
lensname = os.path.basename(filename).split('.')[0].replace('-i', '').replace('-r', '').replace('_alt', '')
print("Selected file: {}".format(lensname))

### Get other infos for the configs

In [ ]:
# fits table extracted from https://www.ast.cam.ac.uk/ioa/research/lensedquasars/index.html
qsos = fits.open("data/lensedquasars.fits")[1]

# JSON extracted from https://arxiv.org/abs/1002.2570
with open('data/table_1002.2570.json') as json_file:
    delay_qsos = json.load(json_file)

In [ ]:
# info from fits table
lensedquasars_table = qsos.data
if 'WFI' in lensname:
    lensname = lensname.replace('J', '')
info_table = lensedquasars_table[lensedquasars_table['Name'] == lensname]
pprint.pprint(zip(info_table.dtype.names, info_table[0]))
table_keys = {'Name': ('name', lambda k: str(k[0])), 'z_lens': ('zl', lambda s: float(s.replace('?', ''))), 'z_qso': ('zs', float)}
info_table = {table_keys[name][0]: table_keys[name][1](info_table[name])
              for name in info_table.dtype.names if name in table_keys}


In [ ]:
if 'WFI' in lensname:
    lensname = lensname.replace('WFI', 'WFIJ')
info_json = delay_qsos[[str(dct['shortname']).replace(" ", "") for dct in delay_qsos].index(lensname)]
positions = np.array([info_json[name] for name in ['x', 'y']]).T
json_keys = {'zl': 'zl', 'zs': 'zs', 'delays': 'delay'}
info_json = {json_keys[name]: info_json[name] for name in info_json.keys() if name in json_keys}
# info_json['name'] = lensname
info_json['srcimgs'] = positions
info_json['tdelay'] = [td[0] if td is not None else 0 for td in info_json['delay']]
info_json['tderr'] = [max(td[1], td[2]) if td is not None else 0 for td in info_json['delay']]
info_json

In [ ]:
info = info_json.copy()
info.update(info_table)
pprint.pprint(info)

### Read fits file

In [ ]:
hdu = fits.open(filename)
hdu[0].header

In [ ]:
lo = LensObject(filename, auto=False, glscfactory_options={'name': info['name'], 'output': info['name']},
                zl=info['zl'], zs=info['zs'],
                tdelay=info['tdelay'], tderr=info['tderr'])
lo.lens = lo.center
for img in info['srcimgs']:
    lo.add_srcimg(lo.lens.xy+img/lo.px2arcsec, unit='pixel', relative=True)
# lo = MultiLens(filename)
print(lo.__v__)

In [ ]:
if 'ORIENTAT' in lo.hdr:
    orientat = lo.hdr['ORIENTAT']
else:
    orientat = 0
orientat

In [ ]:
print(lo.srcimgs_xy)

### Check out lens

In [ ]:
%matplotlib notebook

fig, ax = plt.subplots()

# from matplotlib import transforms
# transf = transforms.Affine2D().rotate_deg(orientat) + ax.transData
# fig, ax, plt_out = lo.plot_f(fig, ax=ax, cmap=gcl.GLEAMcmaps.phoenix, colorbar=True,
#                              vmin=0, vmax=0.5*np.max(lo.data),
#                              lens=True, source_images=True, label_images=False, scalebar=False,
#                              transform=transf)

fig, ax, plt_out = lo.plot_f(fig, ax=ax, cmap=gcl.GLEAMcmaps.vilux, colorbar=True,
                             vmin=0, vmax=0.5*np.max(lo.data), source_images=False,
                             deconv=False, psf=np.ones((3, 3))/25.)
clrbar = IPColorbar(plt_out[1], plt_out[0])
clrbar.connect()
cache = IPPointCache(plt_out[0])
cid = cache.connect()

In [ ]:
if cache.xy:
    subj = cache.modes.index('L')
    print(lo.px2arcsec)
    print(subj)
    d = cache.distances_to(subj, pixel_scale=np.array(lo.px2arcsec))
    pprint.pprint(d)
    pprint.pprint([np.sqrt(di[0]**2 + di[1]**2) for di in d])
    

In [ ]:
%%script false
# pprint.pprint(lo.hdr)
# print("Pixel coordinates")
# lens = lo.lens.xy
# pprint.pprint(lens)
# ABCD = lo.srcimgs_xy
# pprint.pprint(ABCD)

# print("Pixel scale")
# px2arcsec = lo.px2arcsec
# pprint.pprint(px2arcsec)

print("Arcsec coordinates")
arcsecs = np.array(lo.finder.relative_positions(lo.srcimgs, lo.lens))
# pprint.pprint(arcsecs)
pprint.pprint(info['srcimgs'])

# print("Maximal separation")
# pprint.pprint(lo.maprad)
# pprint.pprint([np.sqrt((x+y)**2) for (x, y) in arcsecs])

print("Time delay")
pprint.pprint(lo.tdelay)

### Save GLASS configs

In [ ]:
%%script false
lo.glscfactory.sync_lens_params()
configs = lo.glscfactory.config['single']
print("".join(configs))

In [ ]:
%%script false
savename = "{}.gls".format(lensname)
lo.glscfactory.write(filename=savename)